In [ ]:
%pip install docling quackling llama-index llama-index-llms-openllm
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import PipelineOptions
import logging
import os
from dotenv import load_dotenv

_log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
source = "/home/noelo/dev/instruct-injest/data/CELEX_32021R1173_EN_TXT.pdf"
converter = DocumentConverter(pipeline_options=PipelineOptions())
result = converter.convert_single(source)
print(len(result.pages))

In [ ]:
load_dotenv()
print(os.getenv("API_KEY"))
print(os.getenv("LLM_URL"))

In [ ]:
from quackling.llama_index.node_parsers import HierarchicalJSONNodeParser
from quackling.llama_index.readers import DoclingPDFReader

reader = DoclingPDFReader(parse_type=DoclingPDFReader.ParseType.JSON)
node_parser = HierarchicalJSONNodeParser()

In [ ]:
docs = reader.load_data(file_path=source)

In [ ]:
from rich.pretty import pprint
pprint(docs, max_length=2, max_string=250, max_depth=4)

In [ ]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[node_parser],
)
nodes = pipeline.run(documents=docs)
print(len(nodes))
# for x in nodes:
#     print(x.text)

In [ ]:
raw_text =''
for x in nodes:
    raw_text += x.text

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('nomic-ai/nomic-embed-text-v1.5')
encoded_output = tokenizer(raw_text[:10000])

In [ ]:
print(len(encoded_output.input_ids))

In [ ]:
from llama_index.llms.openllm import OpenLLM  

In [ ]:
from __future__ import annotations

from typing import List, Optional

from pydantic import BaseModel, Field, constr


class QuestionsAndAnswer(BaseModel):
    question: constr(min_length=1)
    answer: constr(min_length=1)


class SeedExample(BaseModel):
    context: constr(min_length=1)
    questions_and_answers: List[QuestionsAndAnswer] = Field(
        ..., min_items=3, set=True
    )


class QNAModel(BaseModel):
    created_by: Optional[constr(min_length=1)] = None
    domain: Optional[constr(min_length=1)] = Field(
        None, examples=['Chemistry', 'History', 'Pop culture']
    )
    seed_examples: Optional[List[SeedExample]] = Field(
        None, min_items=5, set=True
    )


In [ ]:
gen_prompt=f"you are a helpful writing assistant. Given the following context generate 30 question and answer pairs. Ensure that the questions can be answered by the context given.  Group the questions into different semantic contexts. Also output a description of the contexts. You MUST answer using the following json schema: {QNAModel.schema_json()}"

In [ ]:
llm = OpenLLM(
    model="mistral-7b-instruct", api_base=os.getenv("LLM_URL"),
    api_key=os.getenv("API_KEY")
)


# completion_response = llm.complete(gen_prompt+raw_text[:5000],max_tokens=1000,timeout=120.0)
# print(completion_response)

comp_resp=''

for it in llm.stream_complete(gen_prompt+raw_text[:5000],max_tokens=4000,timeout=120.0):
    comp_resp+=it.text


In [ ]:
print(it)

In [ ]:

%pip -q install semantic-router semantic-chunkers

from semantic_router.encoders.fastembed import FastEmbedEncoder

encoder = FastEmbedEncoder()

In [ ]:


from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger

logger.setLevel("WARNING")  # reduce logs from splitter

splitter = RollingWindowSplitter(
    encoder=encoder,
    dynamic_threshold=True,
    min_split_tokens=100,
    max_split_tokens=4000,
    # window_size=2,
    plot_splits=True,  # set this to true to visualize chunking
    enable_statistics=True  # to print chunking stats
)
     

splits = splitter([result.output.export_to_markdown()])

In [ ]:
from semantic_chunkers import StatisticalChunker

chunker = StatisticalChunker(encoder=encoder,enable_statistics=True,plot_chunks=False,min_split_tokens=300, max_split_tokens=2000)

chunks = chunker(docs=[result.output.export_to_markdown()])


In [ ]:
chunker.print(chunks[0])